# GPU Info

In [1]:
!git clone https://github.com/jmhaussaire/SOAI-Project.git
import os
os.chdir('SOAI-Project')
!ls

Cloning into 'SOAI-Project'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 73 (delta 29), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.
common.py		    README.md
CVTracker.py		    Swiss10_prep.ipynb
DetectionAndTrackingCV.py   TrafficCamera_DetectionAndTrackingIOU.ipynb
DetectionAndTrackingIOU.py  TrafficCamera_DetectionAndTracking.ipynb
detectionprovider.py	    video.py
IOUTracker.py		    yolo3
LICENSE			    yolo.py
model_data


In [0]:
!nvidia-smi

# Imports

In [3]:
%reload_ext autoreload
%autoreload 2
from os.path import join
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


Remount drive

In [23]:
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [0]:
!pip install -q pascal-voc-writer

In [7]:
import matplotlib.pyplot as plt
from detectionprovider import DetectionProvider
from common import limit_gpu_memory, Detection, show_LOIs, show_LOI_Info
from IOUTracker import IOUTracker
from video import VideoStreamReader, VideoStreamWriter
from yolo import YOLO
from tqdm import tqdm_notebook
import numpy as np
limit_gpu_memory(0.7)
import cv2
from pascal_voc_writer import Writer as LabelWriter
import csv
import time
from datetime import datetime
import pandas as pd

Using TensorFlow backend.


# Parameters


In [0]:
import os

# YOLO
yolo_score = 0.6
model_data_path = "/gdrive/My Drive/SOAI/TrafficAnalysis-master/model_data/yolo.h5"

# Inputs
data_path = "/gdrive/My Drive/SOAI/TrafficAnalysis-master/"
in_csv_name = os.path.join(data_path,'Vehicles_2016-05-10.csv')
startTime = pd.to_datetime('10.05.2016 11:53:16')
in_vid_name = os.path.join(data_path,"MAH00004.MP4")

# Outputs
out_folder = "/gdrive/My Drive/SOAI/outputs_17_02_2020/"
out_vid_name = os.path.join(out_folder,"900sec.mp4")
seconds_skip = 0  # number of seconds to skip
seconds_count = 30  # number of seconds to process
csv_out = os.path.join(out_folder,'Vehicle-list.csv')

timeInit = 42795.100


# Prepare YOLO network

In [9]:
yolo = YOLO(score=yolo_score, model_path = model_data_path)











/gdrive/My Drive/SOAI/TrafficAnalysis-master/model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Prepare CSV

In [0]:
df = pd.read_csv(in_csv_name)
df['Start'] = pd.to_datetime(df["Start-Timestamp"],format='%m.%d.%Y %H:%M:%S')
df['End'] = pd.to_datetime(df["End-Timestamp"],format='%m.%d.%Y %H:%M:%S')
df_new =  df[df['Start'] >= startTime] 
df_new.reset_index(inplace=True)

# Run through frames

In [14]:
%%time

vidreader = VideoStreamReader(
    in_vid_name, seconds_count=seconds_count, seconds_skip=seconds_skip, width=1920, height=1080)
vidwriter = VideoStreamWriter(
    out_vid_name, width=vidreader.width, height=vidreader.height, fps=vidreader.fps)
detection_provider = DetectionProvider(yolo)

# define the detection zone??
padding = 0
frame_bbox = Detection.from_frame(
    (vidreader.width, vidreader.height), int(padding))

tracker = IOUTracker()

csv_writer = True
videoShowLOI = True
LOIs = [[(450, 400), (600, 850)]]
ovwr = 0
vehicleInVid = 1.5 #when does the vehice first appears?
ovwrInfo = ('-','-','-','-')


# initialize .csv
with open(csv_out, 'w+') as f:
    writer = csv.writer(f)
    csv_line = 'timestamp,lane,vehicle,direction'
    writer.writerows([csv_line.split(',')])
# nice progress bar:
pbar = tqdm_notebook(total=vidreader.frame_count - vidreader.frame_skip)

# main loop
while True:
    frame = vidreader.next_frame()
    if frame is None:
        break
    imgName = "img_%s" % (vidreader.frame_no)
    XMLPathName = os.path.join(out_folder, imgName)
    imgPathName = os.path.join(out_folder, imgName)
    labelwriter = LabelWriter(XMLPathName, vidreader.width, vidreader.height)
    #yolo draws bounding boxes (bb) right into the image.
    #Since I want to be able to save images without bb, I have to make a copy:
    frame_copy = frame.copy()
    captureFrame = False
    captureCSV = False
    detected = False  
    pbar.update()
    
    #yolo detection: 
    detections = detection_provider.detect_boxes(frame, vidreader.frame_no)
    tracker.update(detections.copy(), frame_bbox)

    for track in tracker.active_tracks:
        track.likely_class()
        track.lane_detector()
        #lineDetections = track.csv_detector(LOIs)
        # checking for non-car vehicles that cross the "line of interest":
        for idx, val in enumerate(track.csv_detector(LOIs)):
            if val:
                #print("Detection on: ",idx)
                captureFrame = True
                captureCSV = True
                vTime = vidreader.capture.get(cv2.CAP_PROP_POS_MSEC) / 1000
                nowTime = startTime + \
                    pd.Timedelta(seconds=vTime) - \
                    pd.Timedelta(seconds=vehicleInVid)
                
                # look for a timestamp match in CSV file:
                timeRange = pd.Timedelta(seconds=0.7)
                matchingRows = df_new.loc[(df_new['Start'] > nowTime.floor('S') - timeRange) \
                    & (df_new['Start'] < nowTime.floor('S') + timeRange)]
                #matchingRows = df_new.loc[df_new['Start'].dt.floor(
                #    'S') == nowTime.floor('S')]
                if not matchingRows.empty:
                    probVeh = matchingRows['ExtCatText'].str.lower()
                    for vItem in probVeh.iteritems():
                        if vItem[1] != "car":
                            # overwrite vehicle class with data from CSV:
                            track.ovwrClass = vItem[1]
                    ovwr = vidreader.frame_no + 13
                    ovwrInfo = (track.detections[-1].object_class, track.ovwrClass, track.lane, str(nowTime.floor('S')))
                    #print("found match in csv")
                    
        track.show_history(frame)
        
        if captureCSV:
            if track.ovwrClass is not None:
                csv_line = str(nowTime) + "," + str(track.lane) + "," + \
                    str(track.ovwrClass) + "," + str(track.direction)
            elif track.res is not None:
                csv_line = str(nowTime) + "," + str(track.lane) + \
                    "," + str(track.res) + "," + str(track.direction)
            else:
                print("Hä?")
            with open(csv_out, 'a') as f:
                writer = csv.writer(f)
                (timestamp, lane, vehicle, direction) = csv_line.split(',')
                writer.writerows([csv_line.split(',')])
                #print("writing to csv")
            captureCSV = False

        if captureFrame or (ovwr > vidreader.frame_no and (vidreader.frame_no % 4 == 0)):
            dbox = track.detections[-1].to_tlbr()
            bx1 = dbox[0]
            by1 = dbox[1]
            bx2 = dbox[2]
            by2 = dbox[3]
            if track.res is None:
                print("track.res is None?!", track.res)
                #track.res = track.detections[-1].object_class
            if track.ovwrClass is not None:
                labelwriter.addObject(track.ovwrClass, bx1, by1, bx2, by2)
            elif track.res is not None:
                labelwriter.addObject(track.res, bx1, by1, bx2, by2)
            else:
                labelwriter.addObject("unknown", bx1, by1, bx2, by2)
            labelwriter.save(XMLPathName + ".xml")
            #print("write xml")

    if videoShowLOI:
        vTime = vidreader.capture.get(cv2.CAP_PROP_POS_MSEC) / 1000
        nowTime = startTime + \
                    pd.Timedelta(seconds=vTime) - \
                    pd.Timedelta(seconds=vehicleInVid)
        show_LOIs(frame, LOIs)        
        show_LOI_Info(frame,*ovwrInfo)
        
    
    vidwriter.write(frame)
    
    #save image every 4 frames if non-car is detected
    #or has been detected max. 13 frames before:
    if captureFrame or (ovwr > vidreader.frame_no and (vidreader.frame_no % 4 == 0)):
        cv2.imwrite(imgPathName + ".jpg",
                    cv2.cvtColor(frame_copy, cv2.COLOR_RGB2BGR))
        #print("save image")
        

pbar.close()
vidreader.release()
vidwriter.release()
print("job done")

job done
CPU times: user 30.3 s, sys: 830 ms, total: 31.1 s
Wall time: 36 s


# Functions

In [0]:
def process_vid(in_vid_name,seconds_count,yolo,out_folder):
  vidreader = VideoStreamReader(
    in_vid_name, seconds_count=seconds_count, seconds_skip=seconds_skip, width=1920, height=1080)
  detection_provider = DetectionProvider(yolo)

  # define the detection zone??
  padding = 0
  frame_bbox = Detection.from_frame(
      (vidreader.width, vidreader.height), int(padding))

  tracker = IOUTracker()

  ovwr = 0
  ovwrInfo = ('-','-','-','-')

  # nice progress bar:
  pbar = tqdm_notebook(total=vidreader.frame_count - vidreader.frame_skip)

  # main loop
  while True:
    frame = vidreader.next_frame()
    if frame is None:
        break
    imgName = "img_%s" % (vidreader.frame_no)
    XMLPathName = os.path.join(out_folder, imgName)
    imgPathName = os.path.join(out_folder, imgName)
    labelwriter = LabelWriter(XMLPathName, vidreader.width, vidreader.height)
    #yolo draws bounding boxes (bb) right into the image.
    #Since I want to be able to save images without bb, I have to make a copy:
    frame_copy = frame.copy()
    captureFrame = False
    captureCSV = False
    detected = False  
    pbar.update()
    
    #yolo detection: 
    detections = detection_provider.detect_boxes(frame, vidreader.frame_no)
    tracker.update(detections.copy(), frame_bbox)

    vTime = vidreader.capture.get(cv2.CAP_PROP_POS_MSEC) / 1000
    nowTime = startTime + \
              pd.Timedelta(seconds=vTime)              

          
    for track in tracker.active_tracks:        
        track.likely_class()
        track.lane_detector()
        track.show_history(frame)
                
        csv_line = str(nowTime) + "," + str(track.lane) + \
              "," + str(track.res) + "," + str(track.direction)
      
        dbox = track.detections[-1].to_tlbr()
        bx1 = dbox[0]
        by1 = dbox[1]
        bx2 = dbox[2]
        by2 = dbox[3]
        if track.res is None:
          print("track.res is None?!", track.res)
        if track.res is not None:
          labelwriter.addObject(track.res, bx1, by1, bx2, by2)
        else:
          labelwriter.addObject("unknown", bx1, by1, bx2, by2)
        labelwriter.save(XMLPathName + ".xml")

        with open(csv_out, 'a') as f:
            writer = csv.writer(f)
            (timestamp, lane, vehicle, direction) = csv_line.split(',')
            writer.writerows([csv_line.split(',')])
            #print("writing to csv")
                            

  pbar.close()
  vidreader.release()
  print("job done")


In [32]:
process_vid(in_vid_name,seconds_count,yolo,out_folder)

job done
